In [ ]:
from algorithmV1 import Encoding

import pickle
import numpy as np
import matplotlib.pyplot as plt

from scipy.io.wavfile import read
from scipy.signal import spectrogram
import scipy.signal

fs, s = read("samples\Frisk - Au.Ra.wav")
size = 128
noverlap = 32
# nperseg si la fenêtre n'a pas directement la bonne taille
window = scipy.signal.get_window("boxcar", size, fftbins=True)

### Question 1

In [ ]:
fingerprint = Encoding(window, size)
spectro, peak = fingerprint.process(fs, s)
f, t, Sxx = spectro